<a href="https://colab.research.google.com/github/RolandHewage/RolyEmployeeRetentionPredictor/blob/master/RolyEmployeeRetentionPredictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
df = pd.read_csv("https://raw.githubusercontent.com/mwitiderrick/kerasDO/master/HR_comma_sep.csv")

In [2]:
df.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,department,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [0]:
feats = ['department','salary']
df_final = pd.get_dummies(df,columns=feats,drop_first=True)

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X = df_final.drop(['left'],axis=1).values
y = df_final['left'].values

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [0]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [8]:
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [9]:
classifier = Sequential()

W0830 11:01:50.390137 139934237353856 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



In [10]:
classifier.add(Dense(9, kernel_initializer = "uniform",activation = "relu", input_dim=18))

W0830 11:01:53.689514 139934237353856 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0830 11:01:53.693430 139934237353856 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [0]:
classifier.add(Dense(1, kernel_initializer = "uniform",activation = "sigmoid"))

In [12]:
classifier.compile(optimizer= "adam",loss = "binary_crossentropy",metrics = ["accuracy"])

W0830 11:01:59.356505 139934237353856 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:793: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0830 11:01:59.392166 139934237353856 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3657: The name tf.log is deprecated. Please use tf.math.log instead.

W0830 11:01:59.399363 139934237353856 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [13]:
classifier.fit(X_train, y_train, batch_size = 10, epochs = 1)

W0830 11:02:03.635227 139934237353856 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:1033: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Epoch 1/1
10499/10499 [==============================] - 1s 135us/step - loss: 0.4329 - acc: 0.7995


In [0]:
y_pred = classifier.predict(X_test)


In [0]:
y_pred = (y_pred > 0.5)


In [16]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[3288,  100],
       [ 631,  481]])

In [0]:
new_pred = classifier.predict(sc.transform(np.array([[0.26,0.7 ,3., 238., 6., 0.,0.,0.,0., 0.,0.,0.,0.,0.,1.,0., 0.,1.]])))

In [18]:
new_pred = (new_pred > 0.5)
new_pred

array([[False]])

In [19]:
new_pred = (new_pred > 0.6)
new_pred

array([[False]])

In [0]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

In [0]:
def make_classifier():
    classifier = Sequential()
    classifier.add(Dense(9, kernel_initializer = "uniform", activation = "relu", input_dim=18))
    classifier.add(Dense(1, kernel_initializer = "uniform", activation = "sigmoid"))
    classifier.compile(optimizer= "adam",loss = "binary_crossentropy",metrics = ["accuracy"])
    return classifier

In [0]:
classifier = KerasClassifier(build_fn = make_classifier, batch_size=10, nb_epoch=1)

In [0]:
accuracies = cross_val_score(estimator = classifier,X = X_train,y = y_train,cv = 10,n_jobs = -1)


In [24]:
mean = accuracies.mean()
mean

0.8328378002413002

In [25]:
variance = accuracies.var()
variance

0.0031894569458912224

In [26]:
from keras.layers import Dropout

classifier = Sequential()
classifier.add(Dense(9, kernel_initializer = "uniform", activation = "relu", input_dim=18))
classifier.add(Dropout(rate = 0.1))
classifier.add(Dense(1, kernel_initializer = "uniform", activation = "sigmoid"))
classifier.compile(optimizer= "adam",loss = "binary_crossentropy",metrics = ["accuracy"])

W0830 11:04:23.716876 139934237353856 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3733: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
from sklearn.model_selection import GridSearchCV
def make_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(9, kernel_initializer = "uniform", activation = "relu", input_dim=18))
    classifier.add(Dense(1, kernel_initializer = "uniform", activation = "sigmoid"))
    classifier.compile(optimizer= optimizer,loss = "binary_crossentropy",metrics = ["accuracy"])
    return classifier

In [0]:
classifier = KerasClassifier(build_fn = make_classifier)

In [0]:
params = {
    'batch_size':[20,35],
    'epochs':[2,3],
    'optimizer':['adam','rmsprop']
}

In [0]:
grid_search = GridSearchCV(estimator=classifier,
                           param_grid=params,
                           scoring="accuracy",
                           cv=2)

In [31]:
grid_search = grid_search.fit(X_train,y_train)

Epoch 1/2
5249/5249 [==============================] - 1s 98us/step - loss: 0.5706 - acc: 0.7664
Epoch 2/2
5249/5249 [==============================] - 0s 60us/step - loss: 0.4121 - acc: 0.7664
Epoch 1/2
5250/5250 [==============================] - 1s 96us/step - loss: 0.5908 - acc: 0.7613
Epoch 2/2
5250/5250 [==============================] - 0s 60us/step - loss: 0.4020 - acc: 0.8269
Epoch 1/2
5249/5249 [==============================] - 1s 96us/step - loss: 0.5961 - acc: 0.7643
Epoch 2/2
5249/5249 [==============================] - 0s 59us/step - loss: 0.4403 - acc: 0.7916
Epoch 1/2
5250/5250 [==============================] - 1s 100us/step - loss: 0.5840 - acc: 0.7650
Epoch 2/2
5250/5250 [==============================] - 0s 58us/step - loss: 0.4316 - acc: 0.7882
Epoch 1/3
5249/5249 [==============================] - 1s 115us/step - loss: 0.5699 - acc: 0.7651
Epoch 2/3
5249/5249 [==============================] - 0s 61us/step - loss: 0.3963 - acc: 0.7990
Epoch 3/3
5249/5249 [=======

In [0]:
best_param = grid_search.best_params_
best_accuracy = grid_search.best_score_

In [33]:
best_param

{'batch_size': 20, 'epochs': 3, 'optimizer': 'rmsprop'}

In [34]:
best_accuracy

0.8713210781979236